**Goal** Replicate the Ayagari model by professor; understand the functions we are using and have a mild idea of what is being done. Maybe do the same but try to do it more efficiently. 

In [62]:
#Place the packages here. 
using Printf;
using Roots;
using Plots;
using Interpolations;
using LinearAlgebra;
#wider page display
display("text/html", "<style>.container { width:100% !important; }</style>")

In [63]:
#Code the functions: utility and inverse marginal utility

util= (x, σ)-> x>0 ? (x^(1.0-σ)-1.0)/(1.0-σ) : -Inf
inv_mutil= (x, σ)-> x>0 ? x^(-1.0/σ) : NaN

#This is slightly more generalizabe than his code. 


#59 (generic function with 1 method)

In [64]:
const size_k 	= 1000;    #number of grid points     
const size_m 	= 7;       #number of productivity classes

const kmin 		= 0.0;
const kmax 		= 1700.0;

const β 	= 0.96;
const α 	= 0.36;
const δ 	= 0.08;
const σ 	= 3.0;
const labor 	= 1.0219882;

const epsV 		= 1.0e-8;
const epsdist 	= 1.0e-10;
const epsK		= 1.0e-6;
const relaxsK 	= 0.01;

## grid constants
const scale1 = 1.6;
const grmin = (kmin/scale1)-1.0;
const exponen=log((kmax/scale1)-grmin)/(size_k-1);

const sstates = [exp(-0.600000000000000)	  exp(-0.400000000000000)	  exp(-0.200000000000000)	  exp(0.000000000000000)	   exp(0.200000000000000)	   exp(0.400000000000000)	   exp(0.600000000000000)];

const strans = [	
    0.046746218637144	   0.217937777267117	   0.397822606398702	   0.266386738072197	   0.065169922261456	   0.005754191945237	   0.000182545418147;	
	0.023199661746751	   0.149524091076020	   0.369020347246402	   0.333823905199677	   0.110578117872631	   0.013276146769082	   0.000577730089437;	
	0.010548958644399	   0.093657511915497	   0.312761268311836	   0.382193227897354	   0.171253064028981	   0.027919224002876	   0.001666745199056;	
	0.004387354018187	   0.053538402796357	   0.242163972572887	   0.399820541225137	   0.242163972572887	   0.053538402796357	   0.004387354018187;	
	0.001666745199056	   0.027919224002876	   0.171253064028981	   0.382193227897354	   0.312761268311837	   0.093657511915497	   0.010548958644399;	
	0.000577730089436	   0.013276146769082	   0.110578117872631	   0.333823905199677	   0.369020347246403	   0.149524091076020	   0.023199661746751;	
	0.000182545418147	   0.005754191945237	   0.065169922261456	   0.266386738072197	   0.397822606398702	   0.217937777267117	   0.046746218637144];


#Some adaptations to do inner loop Ayagari
	r    = 0.040237086402090;
	w = (1.0-α)*(α/(r+δ))^(α/(1.0-α))

	param = [β, σ , r , w]
	agrid = (grmin .+ scale1 .* exp.(exponen .* (0:(size_k-1))))
	sgrid = copy(sstates)



	Rgrid = (1 .+ r) .* agrid .+ w .* sgrid

	v0      = util.(Rgrid,σ)./(1.0-β) ##initial h





1000×7 Matrix{Float64}:
  4.81093   6.29323   7.58741   8.68502   9.59039  10.3179  10.8886
  4.94956   6.39391   7.6584    8.73366   9.62283  10.339   10.902
  5.08541   6.49284   7.72834   8.78171   9.65495  10.3599  10.9153
  5.21854   6.59006   7.79725   8.82917   9.68676  10.3807  10.9285
  5.349     6.68558   7.86515   8.87606   9.71825  10.4013  10.9417
  5.47687   6.77946   7.93205   8.92237   9.74943  10.4218  10.9548
  5.6022    6.87171   7.99796   8.96811   9.7803   10.4421  10.9678
  5.72505   6.96237   8.0629    9.01329   9.81087  10.4622  10.9808
  5.84547   7.05147   8.12689   9.05792   9.84113  10.4822  10.9937
  5.96351   7.13904   8.18994   9.102     9.87109  10.502   11.0065
  ⋮                                                 ⋮       
 12.5      12.5      12.5      12.5      12.5      12.5     12.5
 12.5      12.5      12.5      12.5      12.5      12.5     12.5
 12.5      12.5      12.5      12.5      12.5      12.5     12.5
 12.5      12.5      12.5      12.5      

In [65]:
#From last to first steps. 1 derivative at different points of the grid. Here we assume that the function is differentiable, has no kinks etc. 

function deriv(v,k,i)

        λ= (k[i+1]-k[i])/(k[i+1]- k[i-1])
        dv1= ((v[i+1]-v[i]))/(k[i+1]-k[i])
        dv2= ((v[i]-v[i-1]))/(k[i]- k[i-1])
        dv= (1-λ)*dv1 +λ*dv2

end 

#we need to flag which elements in the corner we don't weight; but we can do it all in the same function, no? 

function apply_deriv(v, k)

    Nk, Nz = size(v)
    dv = similar(v)

    for iz in 1:Nz
        V=v[:,iz]
        dv[1,iz]    =(V[2]-V[1])/(k[2]-k[1])
        dv[Nk,iz]   =(V[Nk]-V[Nk-1])/(k[Nk]-k[Nk-1])
        
        for ik in 2:(Nk-1)
            dv[ik, iz]= deriv(V,k,ik) #in between we just take a convex combination (pending to check)
        end 
    end 
    return dv
end

apply_deriv (generic function with 1 method)

In [66]:
function get_assets(c, agrid, sgrid, r, w)

    Na, Ns = size(c)   # (a', s)
    a_endo = similar(c)
    Ga      = similar(c)

    for iz in 1:Ns
        s = sgrid[iz]

        # 1. Endogenous grid for CURRENT asset a_t, here no need to solve for nonlinear function of kapital, straight solution!!!!
        @inbounds for ia in 1:Na
            a_endo[ia, iz] = (c[ia, iz] + agrid[ia] - w*s) ./ (1 + r)
        end
        ##add sanity checks to avoid Na or unsorted values. 

        # 2. Interpolate: map endogenous grid back to exogenous grid
        interp = LinearInterpolation(a_endo[:,iz], agrid; extrapolation_bc = Line())
        Ga[:, iz] = interp.(agrid)
    end

    return a_endo, Ga
end


get_assets (generic function with 1 method)

In [67]:
#Simulation of the value function and interpolation. Once we have the policy function we have to find the fixed point of the value function; using the usual algorithm. 

function expectation(x,P)
    x*P'
end 


function v_simul(v,param,agrid,sgrid,P, r, w)
        Na,Ns = size(v)
        dv   = apply_deriv(v, agrid)
        Ev    = expectation(v,P)
        Edv   = expectation(dv,P)
        ε = 1e-10
        c = inv_mutil.(max.(β .* Edv, ε), σ)
        R     = c.+agrid

        vn= util.(c, σ) +β*Ev
        a_endo, Ga= get_assets(c, agrid, sgrid, r, w)
        Tv=reduce(hcat,[LinearInterpolation(a_endo[:,iz],vn[:,iz];
    extrapolation_bc=Line()).(agrid) for iz in 1:Ns])
        

    return Tv,Ga

end 

v_simul (generic function with 2 methods)

In [68]:
function bell_eq(v,param,agrid,sgrid,P,r, w)
    β, σ = param
    Na,Ns     = size(v)    

    crit= 1
    tol= 1e-08
    iter= 0
    local Tv,Ga
    while crit>tol 
        Tv, Ga = v_simul(v,param,agrid,sgrid,P, r, w)
        crit= norm(Tv-v, Inf)
        v=copy(Tv) #don't occupy as much memory. 
        iter +=1
    end 
    println("Iteration: $iter, Crit: $crit")
    return Tv, Ga
end

bell_eq (generic function with 1 method)

In [69]:
Tv, Ga= v_simul(v0, param, agrid, sgrid, strans)


([6.863316698971184 7.768531805353145 … 10.445496027857681 10.887455295359539; 6.923096943321531 7.814310305211437 … 10.458572385512335 10.896532654631507; … ; 12.499991058433135 12.499991060495576 … 12.499991073152085 12.499991077910735; 12.499991182284667 12.499991184304392 … 12.499991196698856 12.499991201358998], [0.696353270554477 0.7874171112931362 … 1.4335538525579035 1.7012923984123534; 0.7048919757794311 0.7959579969998811 … 1.4421096874571688 1.7098526769972715; … ; 1295.830859494544 1295.921436771457 … 1296.566735203371 1296.8346081708523; 1304.906363152196 1304.9969403500236 … 1305.6422384143525 1305.9101113303698])

In [ ]:
Tv, Ga= bell_eq(v0,param,agrid,sgrid,strans,r, w)

#note: to update, this is currently not working; I think it fails at some iteration of v, the derivative does not work properly (or the invmarg does not return a value). And then the interpolation is done on a vector of Na values, which is one condition that violates the program. Thus, it collapses the whole program. Until v_simul it works wonders.



ErrorException: knot-vectors must be unique and sorted in increasing order